In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import os
import shutil

# Set up Chrome options
chrome_options = Options()
chrome_options.add_experimental_option('prefs', {
    "download.default_directory": r"C:\Users\dhire\OneDrive\Desktop\code\MLB-player-prop-analysis",  # Change this to your preferred download directory
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

# Set up the WebDriver
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open the webpage
url = 'https://www.rotowire.com/baseball/stats.php'
driver.get(url)
time.sleep(3)

download_button = driver.find_element(By.CLASS_NAME, 'export-button.is-csv')
download_button.click()
headers = ['P', 'C', '1B', '2B', '3B', 'SS', 'OF']
for h in headers:
    tab = driver.find_element(By.CSS_SELECTOR, f'div.filter-tab[data-name="{h}"]')
    tab.click()
    time.sleep(5)
    download_button = driver.find_element(By.XPATH, f'//*[@id="stats{h}"]/div[3]/div[2]/button[2]')
    download_button.click()
    time.sleep(2)

# Wait for the file to download
time.sleep(2)  # Adjust the sleep time if necessary

# # Close the driver
driver.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="statsDH"]/div[3]/div[2]/button[2]"}
  (Session info: chrome=125.0.6422.76); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00D2B793+45827]
	(No symbol) [0x00CBDB74]
	(No symbol) [0x00BB150F]
	(No symbol) [0x00BF20BC]
	(No symbol) [0x00BF216B]
	(No symbol) [0x00C2E0F2]
	(No symbol) [0x00C12E44]
	(No symbol) [0x00C2C034]
	(No symbol) [0x00C12B96]
	(No symbol) [0x00BE6998]
	(No symbol) [0x00BE751D]
	GetHandleVerifier [0x00FE43C3+2899763]
	GetHandleVerifier [0x010377ED+3240797]
	GetHandleVerifier [0x00DB1264+593364]
	GetHandleVerifier [0x00DB818C+621820]
	(No symbol) [0x00CC6F54]
	(No symbol) [0x00CC3658]
	(No symbol) [0x00CC37F7]
	(No symbol) [0x00CB58AE]
	BaseThreadInitThunk [0x76437BA9+25]
	RtlInitializeExceptionChain [0x778DBE3B+107]
	RtlClear

In [64]:
from bs4 import BeautifulSoup
import pandas as pd
import requests 
import json

# Load the HTML content
url = 'https://swishanalytics.com/optimus/mlb/batter-vs-pitcher-stats'

# Send a GET request to the webpage
response = requests.get(url)
html_content = response.content

# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')
script_tag = soup.find('script', text=lambda t: t and 'this.playerArray' in t).string

# Extract the JSON-like object from the script tag
json_str = script_tag.split('this.playerArray = ')[1].split('];')[0] + ']'

# Clean the JSON string
json_str = json_str.replace('\\"', '"')  # Replace escaped quotes
json_str = json_str.replace('\\/', '/')  # Replace escaped slashes

# Remove HTML tags within the JSON string
json_str = re.sub(r'<.*?>', '', json_str)

# Escape remaining problematic characters
json_str = re.sub(r'\\', '', json_str)

# Load the JSON-like object as a Python list
try:
    player_data = json.loads(json_str)
except json.JSONDecodeError as e:
    print(f"JSON decoding failed: {e}")
    # Print the problematic part for debugging
    start = max(0, e.pos - 100)
    end = min(len(json_str), e.pos + 100)
    print(f"Problematic part: {json_str[start:end]}")
    raise
df = pd.DataFrame(player_data)


# Save the DataFrame to a CSV file
df.to_csv('batter_vs_pitcher_stats.csv', index=False)

print("Table data has been extracted and saved to batter_vs_pitcher_stats.csv")


Table data has been extracted and saved to batter_vs_pitcher_stats.csv


In [59]:

script_tag = soup.find('script', text=lambda t: t and 'this.playerArray' in t).string

# Extract the JSON-like object from the script tag
json_str = script_tag.split('this.playerArray = ')[1].split('];')[0] + ']'

# Clean the JSON string
json_str = json_str.replace('\\"', '"')  # Replace escaped quotes
json_str = json_str.replace('\\/', '/')  # Replace escaped slashes

# Remove HTML tags within the JSON string
json_str = re.sub(r'<.*?>', '', json_str)

# Escape remaining problematic characters
json_str = re.sub(r'\\', '', json_str)

# Load the JSON-like object as a Python list
try:
    player_data = json.loads(json_str)
except json.JSONDecodeError as e:
    print(f"JSON decoding failed: {e}")
    # Print the problematic part for debugging
    start = max(0, e.pos - 100)
    end = min(len(json_str), e.pos + 100)
    print(f"Problematic part: {json_str[start:end]}")
    raise


In [63]:
df = pd.DataFrame(player_data)
df

,batter,status,batter_team,pitcher,pitcher_team,position,pa,ab,h,singles,...,fd_pos_batter,fd_pos_pitcher,dk_pos_batter,dk_pos_pitcher,ya_pos_batter,ya_pos_pitcher,fpts_fd_ppa,fpts_dk_ppa,bats,p_throws
0,Jose Ramirez,,Guardians,Jose Quintana,Mets,3B,32,29,9,6,...,DH,DH,3B,RP,DH,DH,1.42,1.22,S,L
1,Salvador Perez,,Royals,Tarik Skubal,Tigers,C,21,21,8,3,...,DH,DH,1B,DH,DH,DH,4.05,3.10,R,L
2,Cody Bellinger,,Cubs,Max Fried,Braves,CF,25,23,8,7,...,CF,CF,OF,RP,CF,CF,2.26,1.80,L,L
3,Kyle Tucker,,Astros,Tyler Anderson,Angels,RF,10,9,7,3,...,RF,RF,OF,SP,RF,RF,6.90,5.40,L,L
4,Josh Lowe,,Rays,Brayan Bello,Red Sox,RF,14,14,7,4,...,RF,RF,RF,RF,RF,RF,3.57,2.86,L,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,Mike Tauchman,,Cubs,Max Fried,Braves,RF,0,0,0,0,...,RF,RF,OF,RP,RF,RF,N/A,N/A,L,L
416,Daniel Vogelbach,,Blue Jays,Nick Nastrini,White Sox,DH,0,0,0,0,...,N/A,N/A,1B,N/A,N/A,N/A,N/A,N/A,L,R
417,Matt Carpenter,,Cardinals,John Means,Orioles,DH,0,0,0,0,...,N/A,N/A,1B,SP,N/A,N/A,N/A,N/A,L,L
418,Matt Carpenter,,Cardinals,Kyle Bradish,Orioles,DH,0,0,0,0,...,N/A,N/A,1B,N/A,N/A,N/A,N/A,N/A,L,R
